In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Activation
import numpy as np
import pretty_midi
import os
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# CHANGE PATH AS NECESSARY
PATH = 'music_data'

In [ ]:
def load_midi_files(path):
    """
    Given a filepath, loads all midi files in that filepath into midi objects and returns
    a list of them.
    Arguments:
        path: the path to the list of midi files
    Returns:
        songs: a list of all MIDI objects that were successfully generated
    """
    songs = []

    for folder in os.listdir(path):
        if folder == '..LICENSING':
            continue
        if not os.path.isdir(os.path.join(path, folder)):
            continue
        for file_name in os.listdir(os.path.join(path, folder)):
            try:
                midi = pretty_midi.PrettyMIDI(os.path.join(path, folder, file_name))
                songs.append(midi)
            except Exception as e:
                continue
    return songs

In [ ]:
songs = load_midi_files(PATH)

In [ ]:
from collections import defaultdict

def get_codes_count():
    """
    Returns a dictionary of MIDI instrument codes (i.e. 0 for Grand Piano, etc.) mapped to the amount
    of times they appear in the dataset.

    For example, if 500 songs in the dataset contain a lead vocal (code 85), 85 should map to 500.

    Returns:
        codes_count:
            a list of all codes found mapped to their occurences in the dataset
    """
    program_count = defaultdict(int)
    for song in songs:
        unique_programs = {instrument.program for instrument in song.instruments}
        for program in unique_programs:
            program_count[program] += 1

    return dict(program_count)

codes = get_codes_count()
print(sorted(codes, key=codes.get, reverse=True))

In [ ]:
# Program codes for midi instruments. Add more as necessary for filtering.
program_codes = {
    'VOICE_LEAD': 85,
    'GRAND_PIANO': 0,
}

def filter_data(data):
    """
    Filters the data into melody and accompaniment. Melody is the track with a lead voice program 
    code. If there is no lead voice instrument, ignore the file. The accompaniment is composed of all 
    other tracks.

    Arguments:
        data: the set of all songs 

    Returns: 
        the set of all voice lead tracks for all available songs
    """
    melodies = []
    accompaniments = []
    for midi in data:
        melody = None
        accompaniment = None
        for i in midi.instruments:
            if i.program == program_codes['VOICE_LEAD'] and not melody:
                melody = i.notes
            elif i.program == program_codes['GRAND_PIANO'] and not accompaniment:
                accompaniment = i.notes
        if melody:
            melodies.append(melody)
            if accompaniment == None:
                accompaniment = i.notes
            accompaniments.append(accompaniment)
    return melodies, accompaniments

In [ ]:
melodies, accompaniments = filter_data(songs)

In [ ]:
import pickle

def save_data(data):
    with open('music_data.pkl', 'wb') as f:
        pickle.dump(data, f)

def load_data():
    with open('music_data.pkl', 'rb') as f:
        loaded_data = pickle.load(f)
    return loaded_data

In [ ]:
save_data((melodies, accompaniments))

In [ ]:
# Convert notes to numeric sequences (pitch and timing encoding)
def notes_to_sequences(notes, seq_length=50):
    sequences = []
    for i in range(0, len(notes) - seq_length):
        seq = [(note.pitch, note.start, note.end) for note in notes[i:i+seq_length]]
        sequences.append(seq)
    return sequences

In [ ]:
# Process and convert notes to sequences
seq_length = 50
melody_sequences = [notes_to_sequences(m, seq_length) for m in melodies]
accompaniment_sequences = [notes_to_sequences(a, seq_length) for a in accompaniments]

In [ ]:
# Flatten nested lists and pad sequences
melody_sequences = np.array([seq for sublist in melody_sequences for seq in sublist])
accompaniment_sequences = np.array([seq for sublist in accompaniment_sequences for seq in sublist])

In [ ]:
# Normalize pitches for better training stability
def normalize_sequences(sequences):
    sequences = np.array(sequences)
    pitch_data = sequences[..., 0]
    timing_data = sequences[..., 1:]
    pitch_data = pitch_data / 127.0  # Normalize pitch (MIDI range is 0-127)
    return np.concatenate([pitch_data[..., None], timing_data], axis=-1)

In [ ]:
melody_sequences = normalize_sequences(melody_sequences)
accompaniment_sequences = normalize_sequences(accompaniment_sequences)